In [1]:
import cv2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
data_dir = './dataset/train/'
dataset = pd.read_csv('./dataset/train.csv')

In [ ]:
dataset['image_size'] = dataset['Id'].apply(lambda image_id : cv2.imread(os.path.join(data_dir, image_id + '.jpg')).shape[:2])
dataset['image_total_size'] = dataset['image_size'].apply(lambda x: x[0] * x[1])
dataset = dataset.drop("Pawpularity", axis=1)

In [5]:
def img2hash(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (16, 16))
    avg = gray.mean()
    bi = 1 * (gray > avg)
    return bi

def hamming_distance(a, b):
    a = a.reshape(1,-1)
    b = b.reshape(1,-1)
    # 같은 자리의 값이 서로 다른 것들의 합
    distance = (a !=b).sum()
    return distance

In [ ]:
same_pic = []
pbar = tqdm(dataset.iterrows(), total=len(dataset), desc='test', mininterval=5)
for index, data in pbar:
    for index2, data2 in dataset[index+1:].iterrows():
        if np.all(data.drop('Id') == data2.drop('Id')):
            img = cv2.imread(os.path.join(data_dir, data['Id'] + '.jpg'))
            img_hash = img2hash(img)
            img2 = cv2.imread(os.path.join(data_dir, data2['Id'] + '.jpg'))
            img2_hash = img2hash(img2)
            dist = hamming_distance(img_hash, img2_hash)
            if dist == 0:
                same_pic.append((data['Id'], data2['Id']))